# Extraindo dados

## Imports

<p>Primeiro importamos algumas bibliotecas que usaremos.</p>

In [1]:
import requests
import json
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

<p>
    A minha ideia inicial foi praticar a extração de dados via API. Fazendo uma busca na internet, encontrei o site <a href="https://openweathermap.org/api" target="_blank">Weather API</a> que disponibiliza várias API's para acessar dados relacionados ao clima. Escolhi aquela que fornece informações atuais de qualquer localidade da Terra (que é redonda à propósito).
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/e3Fh-MnIQWgAAAAC/olavo-terra-plana-olavo-de-carvalho-terraplanista.gif"
    height="400"
    alt="Um trouxa com cara de bobo"><br>
  <div align="center"><sub><figcaption>Olavilson chocado:  "É o quê???"</figcaption></sub></div>
</figure>
<br>

<p>
    OK. Buscar dados referentes a uma única cidade me parecia um pouco sem graça, então decidi buscar de várias e me surgiu a dúvida: quais cidades?
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/6ZT1sLiE8O0AAAAM/esbritpinedo-gabriel-esbritpinedo.gif"
    height="400"
    alt="Um cara perdido tentando se achar com um mapa"><br>
  <div align="center"><sub><figcaption>"Eu sei exatamente para onde ir".</figcaption></sub></div>
</figure>
<br>

<p>
    Já sei! Cidades do estado de SP (estado onde moro).<br><br>Segundo a <a href="https://en.wikipedia.org/wiki/List_of_municipalities_in_the_state_of_S%C3%A3o_Paulo_by_population" target="_blank">Wikipedia</a>, são 645 cidades ao todo. E como uma boa pessoa indecisa: <i>"Na dúvida de qual escolher? Fique com todas"</i>. 
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/Jl1GCaxBFIIAAAAC/qvc-qvc-italia.gif"
    height="400"
    alt="Uma mulher indecisa entre dois vestidos"><br>
  <div align="center"><sub><figcaption>"Vou ficar com as duas".</figcaption></sub></div>
</figure>
<br>

<p>
    Beleza! Então decidi criar um dataset com dados relativos ao clima. Basta eu ter uma lista com o nome de todos as cidades de SP que um <i>loop</i> <code>for</code> da conta dessa tarefa. Simples! É só eu dar um ctrl+C ctrl+V na tabela da Wikipedia que tá fácil.<br><br> Mas tá simples demais, né? Vamos complicar um pouco mais...
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/_FIgEJYgxQMAAAAM/face-palm-panda.gif"
    height="400"
    alt="Um panda lamentando"><br>
  <div align="center"><sub><figcaption>Ahhh nããão...!</figcaption></sub></div>
</figure>
<br>

<p>
    Já que a ideia é praticar a extração de dados e como eu já andei praticando o <a href="https://github.com/ViniViniAntunes/Praticando_Web_Scrap">web scrap</a>, por que não aproveitar extrair os nomes das cidades usando essa técnica?
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/1dMqXacMGJsAAAAM/good.gif"
    height="400"
    alt="House curtindo a ideia"><br>
  <div align="center"><sub><figcaption>BOA!</figcaption></sub></div>
</figure>
<br>

<p>
Pronto! Então vamos começar!
</p>

<p>
    Primeiro, fiz meu cadastro no site para poder ter acesso à API.
</p>

<p>
    Com uma conta <a href="https://openweathermap.org/price" target="_blank"><i>free</i></a>, podemos fazer até <code>1M calls/mês</code> tendo um limite de <code>60 calls/minuto</code>. O que, para o meu propósito, é mais do que suficente.
</p>
<p>
    Após um rápido cadastro, voltei à <a href="https://openweathermap.org/current" target="_blank">documentação</a> da API e lá pude ver que existe a possibilidade de buscar esses dados por nome da cidade, por ID da cidade, pelas coordenadas geográficas ou pelo ZIP Code. Escolhi fazer pelo <u>nome da cidade</u>. Vi também quais parâmetros necessários para fazer as chamadas. São eles:
</p>
<p>
    <ul>
        <li> <code> q </code> (obrigatório) - aqui precisamos colocar o nome da cidade ou cidade e/ou código do estado e/ou código do país </li><br>
        <li> <code> appid </code> (obrigatório) - é a chave da API fornecida ao realizar a conta seja ela gratuita ou paga </li><br>
        <li> <code> mode </code> (opicional) - podemos escolher o formato da resposta da chamada que faremos. Pode ser  <code> html </code>,  <code> xml </code> ou  <code> JSON </code> sendo esse último o padrão caso não coloquemos nada.</li><br>
        <li> <code> units </code> (opicional) - aqui escolhemos a unidade de medida utilizada na saída. Por exemplo, se quisermos a temperatura em <i>Kelvin</i>, colocamos  <code> standard </code> e para graus <i>Celsius</i>, usamos <code> metric </code> para esse parâmetro. Agora, se você gosta de sofrer, use <code> imperial </code> aqui para obter a temperatura em <i>Fahrenheit</i></li><br>
        <li> <code> lang </code> (opicional) - podemos escolher também o idioma que aparecerá na descrição do clima atual na resposta da API.</li><br>
    </ul>
</p>
<p>
    Agora vamos ao código.
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/lBp_H83I7AEAAAAM/morpheus-matrix-fight.gif"
    height="400"
    alt="Morpheus te chamando para o código"><br>
  <div align="center"><sub><figcaption>Partiu?</figcaption></sub></div>
</figure>
<br>


## Funções auxiliares

<p>
    Criei algumas funções para me auxiliar.
</p>

In [2]:
def verifica_request(req):
    '''
    Essa função verifica se a request deu certo e, caso contrário, imprime o código, a razão e a mensagem do erro.
    
    Parâmetros
    ----------
    
    req : objeto requests.models.Response --> retorno do método requests.request()
    
    Retorno
    ----------
    Não há retorno.
    .
    '''
    print(req.url)
    # Verificando se a request deu certo
    if req.status_code != requests.codes.ok:
        erro = f'''
        Error code: {req.status_code}
        Reason: {req.reason}
        Message: {req.content}
        '''
        raise ValueError(erro)
        
    else:
        print('OK...')

def busca_html(url):
    '''
    Essa função verifica se a request deu certo e, caso contrário, imprime o código, a razão e a mensagem do erro.
    
    Parâmetros
    ----------
    
    url : str --> url da página 
    
    Retorno
    ----------
    Retorna um objeto 'bs4.BeautifulSoup' contendo o conteúdo html da página buscada.
    
    .
    '''
    headers = {
        'User-Agent': UserAgent().random
    }

    req = requests.request(method='GET', url=url, headers=headers)
    verifica_request(req)
    
    # Instanciando um objeto BeautifulSoup
    soup = BeautifulSoup(req.text, 'html.parser')
    return soup

## Buscando os nomes das cidades

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_municipalities_in_the_state_of_São_Paulo_by_population'
soup = busca_html(url)

https://en.wikipedia.org/wiki/List_of_municipalities_in_the_state_of_S%C3%A3o_Paulo_by_population
OK...


In [4]:
%%time

tabela = soup.findAll('table')[1]
# Pegando os nomes das colunas da tabela
colunas_names = [col.getText() for col in tabela.find('tbody').findAll('th')]
colunas = {i: col.getText() for i, col in enumerate(tabela.find('tbody').findAll('th'))}

# Criando um DataFrame com os nomes das colunas
cidades = pd.DataFrame(columns=colunas_names)

# Pegando os dados da tabela por linha
for i in range(len(tabela.find('tbody').findAll('tr'))):
    if i != 0:
        linha = tabela.find('tbody').findAll('tr')[i].getText().split('\n')[1:]
        inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
        cidades = pd.concat([cidades, inserir_linha], ignore_index=True)

new_cols = {
    'Sequence\n': 'Sequence',
    'Population 2010\n': 'Municipality'
}

# Tratando alguns detalhes
cidades = cidades.rename(columns=new_cols)[list(new_cols.values())]
cidades['Municipality'] = cidades['Municipality'].apply(lambda x: x.replace('  ', ' '))

# É tetra!
cidades

Wall time: 2.71 s


,Sequence,Municipality
0,1º,São Paulo
1,2º,Guarulhos
2,3º,Campinas
3,4º,São Bernardo do Campo
4,5º,São José dos Campos
...,...,...
640,641º,Santana da Ponte Pensa
641,642º,Flora Rica
642,643º,Nova Castilho
643,644º,Uru


In [5]:
API_key = 'CHAVE_DA_API'

In [6]:
def current_wheather(city_name):
    '''
    Essa função faz uma chamada na API, e retorna um dicionário Python.
    
    Parâmetros
    ----------
    
    city_name : str --> Nome da cidade
    
    Retorno
    ----------
    Retorna um um dicionário Python com o conteúdo da resposta da API.
    
    .
    '''
    
    url = f'http://api.openweathermap.org/data/2.5/weather'
    headers = {
        'User-Agent': UserAgent().random
    }
    
    params = {
        'q': city_name,
        'appid': API_key,
        'mode': 'json',
        'units': 'metric',
        'lang': 'pt_br'
    }
    
    r = requests.request('GET', url, headers=headers, params=params)
    verifica_request(r)
    
    return json.loads(r.content)

<p>
    Vamos testar pra ver que tá tudo OK até aqui. Vamos testar com a cidade de <b>Acapulco</b>.
</p>

<br>
<figure>
  <p><img src="https://i.makeagif.com/media/11-08-2015/FimTj0.gif"
    height="400"
    alt="Seu Madruga e Chiquinha comemorando que vão para Acapulco"><br>
  <div align="center"><sub><figcaption>"Acapulco me espera!!!".</figcaption></sub></div>
</figure>
<br>

In [7]:
r = current_wheather(city_name='Guarujá')

http://api.openweathermap.org/data/2.5/weather?q=Guaruj%C3%A1&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...


<p>
    Entendedores entenderão.
</p>

<br>
<figure>
  <p><img src="http://www.acessogeek.com/wp-content/uploads/2016/05/cap.gif"
    height="400"
    alt="Capitão América entendendo a referência"><br>
  <div align="center"><sub><figcaption>"Eu peguei a referência".</figcaption></sub></div>
</figure>
<br>

<p>
    Foi! Agora vamos ver o que a API nos retornou.
</p>

In [8]:
for chave in r.keys():
    print(f'{chave} --> {r[chave]}', end='\n\n')

coord --> {'lon': -46.2564, 'lat': -23.9931}

weather --> [{'id': 701, 'main': 'Mist', 'description': 'névoa', 'icon': '50n'}, {'id': 300, 'main': 'Drizzle', 'description': 'garoa de leve intensidade', 'icon': '09n'}]

base --> stations

main --> {'temp': 20, 'feels_like': 20.35, 'temp_min': 19.92, 'temp_max': 22.78, 'pressure': 1017, 'humidity': 88}

visibility --> 3000

wind --> {'speed': 3.09, 'deg': 220}

clouds --> {'all': 90}

dt --> 1632261432

sys --> {'type': 1, 'id': 8432, 'country': 'BR', 'sunrise': 1632214528, 'sunset': 1632258041}

timezone --> -10800

id --> 3461789

name --> Guarujá

cod --> 200



<p>
    Sabendo qual é a cara da resposta que a API  nos dá, podemos criar uma lógica para pegar os dados de todas as cidades.</p>

<br>
<figure>
  <p><img src="https://miro.medium.com/max/1000/1*5qaDBed44uCE2L87yeH6pg.gif"
    height="400"
    alt="Imagem representativa de 'agora é muita coisa'"><br>
  <div align="center"><sub><figcaption>Agora o bicho pega.</figcaption></sub></div>
</figure>
<br>

## Buscando os dados de todas as cidades

<p>
    Essa parte é a mais demorada.
</p>
<p>
    Claro! Não era pra menos, são 645 cidades, lembra? E como precisamos respeitar o limite da API de <code>60 calls/minuto</code>, o tempo vai ser maior ainda, já que precisamos esperar entre uma chamada e outra para que não exceda esse limite.
</p>

<p>
    Respira fundo e vai...
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/WlsGWxW7STEAAAAM/cristiano-ronaldo-ronaldo-portugal.gif"
    height="400"
    alt="CR7 cansado"><br>
  <div align="center"><sub><figcaption>Vamos lá...</figcaption></sub></div>
</figure>
<br>

<p>
    Coloquei pra rodar à noite e fui dormir. No dia seguinte, vi que meu programa retornou um erro.
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/FcVg5W9zZJQAAAAM/error.gif"
    height="400"
    alt="Um gatinho desapontado com o erro"><br>
  <div align="center"><sub><figcaption>¬¬</figcaption></sub></div>
</figure>
<br>

<p>
    O motivo foi que alguma cidade na lista com o nome composto estava escrito com 2 espaços entre as palavras e não 1 como seria o certo. Por exemplo: São  João  do  Pau  d'Alho ao invés de <a href="https://goo.gl/maps/29DRZqTCYzrvBNM79" target="_blank">São João do Pau d'Alho</a>. Com isso a API não reconheceu o nome da cidade e retornou um erro.
</p>

<p>
    Para corrigir esse problema, fui direto minha na fonte e acrescentei lá no código do web scrap a função <code>lambda x: x.replace('  ', ' ')</code> que itera sobre todos os nomes da lista e substitui os 2 espaços por apenas 1.
</p>

<p>
    Na noite seguinte, botei pra rodar novamente e acordei no outro dia com outro erro.
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/6V2qHcyxvIEAAAAC/tenor.gif"
    height="400"
    alt="Macaquinho não acreditando que isso está acontecendo"><br>
  <div align="center"><sub><figcaption>(...)</figcaption></sub></div>
</figure>
<br>

<p>
    Dessa vez, foi que a API não encontrou dados para a 375ª cidade da lista, mais precisamente a cidade de <a href="https://goo.gl/maps/3ABHQzyyLZyDbsUT7" target="_blank">Sarapuí</a>. Sigo sem saber o porquê desse erro.
</p>

<p>
    Para contornar esse problema, acrescentei no código abaixo um par <code>try/except</code> que tenta rodar um trecho de código e, caso retorne algum erro inesperado, roda outro trecho sem travar como antes.
</p>

<p>
    Dessa vez não consegui me aguentar de curiosidade e botei pra rodar durante meu horário de almoço no trabalho. E após 37min, deu tudo certo.
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/ehpSU7BMZLMAAAAC/tetra-comemoracao.gif"
    height="400"
    alt="Um momento com uma emoção sem tamanho"><br>
  <div align="center"><sub><figcaption>"É tééétraaAA!!!</figcaption></sub></div>
</figure>
<br>

In [9]:
%%time

inicio = datetime.datetime.strptime('1/1/1970 00:00', '%d/%m/%Y %H:%M')


df = pd.DataFrame()

for cidade in cidades['Municipality']:
    
    print(cidade)
    try:
        r = current_wheather(city_name=cidade)

        dados = {
            'quando': inicio + datetime.timedelta(seconds=r['dt'] + r['timezone']),
            'cidade': r['name'],
            'pais': r['sys']['country'],
            'nascer_do_sol': inicio + datetime.timedelta(seconds=r['sys']['sunrise'] + r['timezone']),
            'por_do_sol': inicio + datetime.timedelta(seconds=r['sys']['sunset'] + r['timezone']),
            'velocidade_vento': r['wind']['speed'],
            'ang_vento': r['wind']['deg'],
            'visibilidade': r['visibility'],
            'temperatura_min': r['main']['temp_min'],
            'temperatura_max': r['main']['temp_max'],
            'pressao': r['main']['pressure'],
            'umidade': r['main']['humidity'],
            'sensacao_termica': r['main']['feels_like'],
            'condicao_meteorologica': r['weather'][0]['main'],
            'descricao': r['weather'][0]['description'],
            'longitude': r['coord']['lon'],
            'latitude': r['coord']['lat']    
        }

        print(dados, end=f'\n\n{"---"*30}\n\n')
        time.sleep(3)

        df = pd.concat([df, pd.DataFrame([dados])], ignore_index=True)
    except:
        dados = {
            'quando': 'dado_indisponivel',
            'cidade': cidade,
            'pais': 'BR',
            'nascer_do_sol': 'dado_indisponivel',
            'por_do_sol': 'dado_indisponivel',
            'velocidade_vento': 'dado_indisponivel',
            'ang_vento': 'dado_indisponivel',
            'visibilidade': 'dado_indisponivel',
            'temperatura_min': 'dado_indisponivel',
            'temperatura_max': 'dado_indisponivel',
            'pressao': 'dado_indisponivel',
            'umidade': 'dado_indisponivel',
            'sensacao_termica': 'dado_indisponivel',
            'condicao_meteorologica': 'dados_indisponivel',
            'descricao': 'dado_indisponivel',
            'longitude': 'dado_indisponivel',
            'latitude': 'dado_indisponivel'
        }

        print(dados, end=f'\n\n{"---"*30}\n\n')
        time.sleep(3)

        df = pd.concat([df, pd.DataFrame([dados])], ignore_index=True)

print('Fim!')

São Paulo
http://api.openweathermap.org/data/2.5/weather?q=S%C3%A3o+Paulo&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 57, 8), 'cidade': 'São Paulo', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 5, 56, 58), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 2, 12), 'velocidade_vento': 5.14, 'ang_vento': 140, 'visibilidade': 4000, 'temperatura_min': 15.08, 'temperatura_max': 17.94, 'pressao': 1019, 'umidade': 90, 'sensacao_termica': 17.04, 'condicao_meteorologica': 'Mist', 'descricao': 'névoa', 'longitude': -46.6361, 'latitude': -23.5475}

------------------------------------------------------------------------------------------

Guarulhos
http://api.openweathermap.org/data/2.5/weather?q=Guarulhos&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 56, 10), 'cidade': 'Guarulhos', 'pais': 'BR', 'nascer_do_sol': datetime.da

Santos
http://api.openweathermap.org/data/2.5/weather?q=Santos&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 53, 13), 'cidade': 'Santos', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 5, 55, 46), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 0, 59), 'velocidade_vento': 3.09, 'ang_vento': 220, 'visibilidade': 3000, 'temperatura_min': 19.95, 'temperatura_max': 22.81, 'pressao': 1017, 'umidade': 88, 'sensacao_termica': 20.44, 'condicao_meteorologica': 'Mist', 'descricao': 'névoa', 'longitude': -46.3336, 'latitude': -23.9608}

------------------------------------------------------------------------------------------

Diadema
http://api.openweathermap.org/data/2.5/weather?q=Diadema&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 57, 58), 'cidade': 'Diadema', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21,

Limeira
http://api.openweathermap.org/data/2.5/weather?q=Limeira&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 58, 36), 'cidade': 'Limeira', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 0, 1), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 5, 17), 'velocidade_vento': 7.56, 'ang_vento': 145, 'visibilidade': 10000, 'temperatura_min': 21.59, 'temperatura_max': 33.22, 'pressao': 1012, 'umidade': 66, 'sensacao_termica': 21.53, 'condicao_meteorologica': 'Clouds', 'descricao': 'nublado', 'longitude': -47.4017, 'latitude': -22.5647}

------------------------------------------------------------------------------------------

Suzano
http://api.openweathermap.org/data/2.5/weather?q=Suzano&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 58, 40), 'cidade': 'Suzano', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 

Araraquara
http://api.openweathermap.org/data/2.5/weather?q=Araraquara&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 59, 18), 'cidade': 'Araraquara', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 3, 5), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 8, 24), 'velocidade_vento': 3.04, 'ang_vento': 200, 'visibilidade': 10000, 'temperatura_min': 31.69, 'temperatura_max': 31.69, 'pressao': 1008, 'umidade': 23, 'sensacao_termica': 29.86, 'condicao_meteorologica': 'Clouds', 'descricao': 'nublado', 'longitude': -48.1756, 'latitude': -21.7944}

------------------------------------------------------------------------------------------

Jacareí
http://api.openweathermap.org/data/2.5/weather?q=Jacare%C3%AD&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 59, 22), 'cidade': 'Jacareí', 'pais': 'BR', 'nascer_do_sol': datetime.d

Bragança Paulista
http://api.openweathermap.org/data/2.5/weather?q=Bragan%C3%A7a+Paulista&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 18, 59, 56), 'cidade': 'Bragança Paulista', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 5, 56, 38), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 1, 54), 'velocidade_vento': 6.99, 'ang_vento': 136, 'visibilidade': 10000, 'temperatura_min': 16.12, 'temperatura_max': 19.69, 'pressao': 1015, 'umidade': 82, 'sensacao_termica': 17.77, 'condicao_meteorologica': 'Clouds', 'descricao': 'nublado', 'longitude': -46.5556, 'latitude': -22.94}

------------------------------------------------------------------------------------------

Pindamonhangaba
http://api.openweathermap.org/data/2.5/weather?q=Pindamonhangaba&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 0), 'cidade': 'Pindamonhangaba', '

...

Florínea
http://api.openweathermap.org/data/2.5/weather?q=Flor%C3%ADnea&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 32, 9), 'cidade': 'Florínea', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 13, 19), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 18, 35), 'velocidade_vento': 1.86, 'ang_vento': 183, 'visibilidade': 10000, 'temperatura_min': 30.05, 'temperatura_max': 30.05, 'pressao': 1008, 'umidade': 22, 'sensacao_termica': 28.35, 'condicao_meteorologica': 'Clouds', 'descricao': 'nublado', 'longitude': -50.7268, 'latitude': -22.9016}

------------------------------------------------------------------------------------------

Timburi
http://api.openweathermap.org/data/2.5/weather?q=Timburi&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 32, 13), 'cidade': 'Timburi', 'pais': 'BR', 'nascer_do_sol': datetime.datet

Arapeí
http://api.openweathermap.org/data/2.5/weather?q=Arape%C3%AD&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 32, 48), 'cidade': 'Arapeí', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 5, 48, 13), 'por_do_sol': datetime.datetime(2021, 9, 21, 17, 53, 28), 'velocidade_vento': 2.32, 'ang_vento': 166, 'visibilidade': 9527, 'temperatura_min': 20.33, 'temperatura_max': 20.33, 'pressao': 1017, 'umidade': 91, 'sensacao_termica': 20.79, 'condicao_meteorologica': 'Clouds', 'descricao': 'nublado', 'longitude': -44.4478, 'latitude': -22.6739}

------------------------------------------------------------------------------------------

Embaúba
http://api.openweathermap.org/data/2.5/weather?q=Emba%C3%BAba&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 32, 52), 'cidade': 'Embaúba', 'pais': 'BR', 'nascer_do_sol': datetime.dateti

http://api.openweathermap.org/data/2.5/weather?q=Dolcin%C3%B3polis&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 33, 27), 'cidade': 'Dolcinópolis', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 12, 24), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 17, 48), 'velocidade_vento': 1.13, 'ang_vento': 156, 'visibilidade': 10000, 'temperatura_min': 30.25, 'temperatura_max': 30.25, 'pressao': 1008, 'umidade': 16, 'sensacao_termica': 28.35, 'condicao_meteorologica': 'Clear', 'descricao': 'céu limpo', 'longitude': -50.5133, 'latitude': -20.1231}

------------------------------------------------------------------------------------------

Santa Clara d'Oeste
http://api.openweathermap.org/data/2.5/weather?q=Santa+Clara+d%27Oeste&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 33, 30), 'cidade': "Santa Clara d'Oeste", 'pais'

Vitória Brasil
http://api.openweathermap.org/data/2.5/weather?q=Vit%C3%B3ria+Brasil&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 34, 5), 'cidade': 'Vitória Brasil', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 12, 17), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 17, 41), 'velocidade_vento': 1.22, 'ang_vento': 167, 'visibilidade': 10000, 'temperatura_min': 30.14, 'temperatura_max': 30.14, 'pressao': 1008, 'umidade': 16, 'sensacao_termica': 28.26, 'condicao_meteorologica': 'Clear', 'descricao': 'céu limpo', 'longitude': -50.4848, 'latitude': -20.1979}

------------------------------------------------------------------------------------------

Paulistânia
http://api.openweathermap.org/data/2.5/weather?q=Paulist%C3%A2nia&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 34, 9), 'cidade': 'Paulistânia', 'pais': 'B

Uru
http://api.openweathermap.org/data/2.5/weather?q=Uru&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 34, 47), 'cidade': 'Uru', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6, 7, 31), 'por_do_sol': datetime.datetime(2021, 9, 21, 18, 12, 50), 'velocidade_vento': 2.67, 'ang_vento': 215, 'visibilidade': 10000, 'temperatura_min': 35.1, 'temperatura_max': 35.1, 'pressao': 1008, 'umidade': 19, 'sensacao_termica': 33.04, 'condicao_meteorologica': 'Clouds', 'descricao': 'nuvens dispersas', 'longitude': -49.282, 'latitude': -21.7846}

------------------------------------------------------------------------------------------

Borá
http://api.openweathermap.org/data/2.5/weather?q=Bor%C3%A1&appid=6a024c0dca560ae1cd7f3d5d776b0169&mode=json&units=metric&lang=pt_br
OK...
{'quando': datetime.datetime(2021, 9, 21, 19, 34, 51), 'cidade': 'Borá', 'pais': 'BR', 'nascer_do_sol': datetime.datetime(2021, 9, 21, 6

<p>
    E aí estão to dados de clima de todas as cidades do estado de SP (pelo menos todas as cidades em que a API tinha os dados).
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/HSf2xZzzDfwAAAAM/excited-hockey.gif"
    height="400"
    alt="Criança comemorando muito'"><br>
  <div align="center"><sub><figcaption>"Uhul...!".</figcaption></sub></div>
</figure>
<br>

In [10]:
df

,quando,cidade,pais,nascer_do_sol,por_do_sol,velocidade_vento,ang_vento,visibilidade,temperatura_min,temperatura_max,pressao,umidade,sensacao_termica,condicao_meteorologica,descricao,longitude,latitude
0,2021-09-21 18:57:08,São Paulo,BR,2021-09-21 05:56:58,2021-09-21 18:02:12,5.14,140,4000,15.08,17.94,1019,90,17.04,Mist,névoa,-46.6361,-23.5475
1,2021-09-21 18:56:10,Guarulhos,BR,2021-09-21 05:56:34,2021-09-21 18:01:48,3.6,180,10000,15.03,17.88,1019,88,17.2,Mist,névoa,-46.5333,-23.4628
2,2021-09-21 18:55:30,Campinas,BR,2021-09-21 05:58:40,2021-09-21 18:03:55,9.26,130,10000,20.89,20.89,1015,78,21.07,Clouds,nuvens dispersas,-47.0608,-22.9056
3,2021-09-21 18:54:34,São Bernardo do Campo,BR,2021-09-21 05:56:42,2021-09-21 18:01:55,7.72,180,5000,14.84,17.69,1019,91,16.64,Mist,névoa,-46.565,-23.6939
4,2021-09-21 18:54:20,São José dos Campos,BR,2021-09-21 05:53:58,2021-09-21 17:59:13,3.6,190,8000,19.04,21.92,1019,77,19.69,Clouds,nublado,-45.8869,-23.1794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640,2021-09-21 19:34:37,Santana da Ponte Pensa,BR,2021-09-21 06:13:32,2021-09-21 18:18:56,2.33,172,10000,30.49,30.49,1008,16,28.54,Clear,céu limpo,-50.7976,-20.2544
641,2021-09-21 19:34:40,Flora Rica,BR,2021-09-21 06:15:54,2021-09-21 18:21:14,2.25,176,10000,33.37,33.37,1007,17,31.11,Clouds,nublado,-51.3805,-21.676
642,2021-09-21 19:34:44,Nova Castilho,BR,2021-09-21 06:11:44,2021-09-21 18:17:06,1.1,239,10000,31.09,31.09,1007,16,29.04,Clouds,nuvens dispersas,-50.3427,-20.7638
643,2021-09-21 19:34:47,Uru,BR,2021-09-21 06:07:31,2021-09-21 18:12:50,2.67,215,10000,35.1,35.1,1008,19,33.04,Clouds,nuvens dispersas,-49.282,-21.7846


<p>
    Essas são as cidades que a API não retornou nada.
</p>

In [11]:
list(df[df['descricao'] == 'dado_indisponivel']['cidade'].values)

['Sarapuí',
 'Santo Antônio do Aracanguá',
 'Icém',
 'Barra do Turvo',
 'Marabá Paulista',
 'Bom Sucesso de Itararé']

## Exportando os dados

<p>
    Para não precisarmos rodar isso tudo outra vez, vamos salvar esses dados num arquivo <code>csv</code>.
</p>

In [12]:
df.to_csv('data//dados_clima_SP.csv', sep='|', index=False)

## Concluindo

<p>
    Com isso, concluímos o nosso objetivo de criar um dataset com dados de clima de todas as cidades do estado de SP. Para isso praticamos a extração de dados por duas fontes distintas, via API e via web utilizando o web scrap. Com isso, por exemplo, poderíamos agendar um job que rode o scrip que busca os dados do clima de uma ou mais cidades com uma frequência maior, registrando tudo em algum banco de dados para uma análise posterior.
</p>

<br>
<figure>
  <p><img src="https://c.tenor.com/OIUrtDM8vu4AAAAC/excited-kid.gif"
    height="400"
    alt="Uma garotinha muito empolgada"><br>
  <div align="center"><sub><figcaption>"Yeah!!!".</figcaption></sub></div>
</figure>
<br>